# TinyNoiseGru Training Notebook

Train a lightweight, causal GRU-based Noise Power Spectrum Estimator suitable for real-time hearing-aid applications.

**Architecture Design:**
- Based on TinyGRUVAD architecture (proven lightweight & causal)
- Outputs noise power spectrum (mel bands) instead of binary VAD
- ~2k parameters for efficient on-device deployment
- Frame-level noise estimation with causal processing

**Target Application:**
- Real-time noise estimation for hearing-aid speech enhancement
- Works with Wiener filtering, Spectral Subtraction.
- Replaces traditional minimum statistics noise estimation

In [ ]:
#!pip install -r ../../requirements.txt

In [ ]:
# %% ------------------------- Cell 1: Imports & Setup ------------------------
import sys
from pathlib import Path
import torch, torch.nn as nn, torch.nn.functional as F
import torchaudio, numpy as np, random
from tqdm.notebook import tqdm

SEED = 0
torch.manual_seed(SEED); np.random.seed(SEED); random.seed(SEED)

current_dir = Path.cwd()
repo_root = current_dir.parent.parent
sys.path.insert(0, str(repo_root / "src"))

print("Repo root:", repo_root)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# helper utils
from utils.audio_dataset_loader import (
    load_ears_dataset, load_wham_dataset, load_noizeus_dataset,
    create_audio_pairs, preprocess_audio
)

In [ ]:
# %% ------------------------- Cell 2: Global Plot Settings -------------------------
import matplotlib.pyplot as plt

# Set global plot parameters
plt.rcParams['figure.dpi'] = 400
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 18
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 18

print("[INFO] Global plot settings applied: DPI=400, Font=Times New Roman 18pt")

In [ ]:
# %% ------------------------- Cell 3: TinyNoiseGru Model -------------------------
class TinyNoiseGru(nn.Module):
    """Lightweight GRU-based Noise Power Spectrum Estimator.
    
    Args:
        input_dim: Number of input features (default: 32 mel bands)
        hidden_dim: GRU hidden size (default: 16, trade-off between capacity & efficiency)
        dropout: Dropout rate (default: 0.1)
    """
    def __init__(self, input_dim=32, hidden_dim=16, dropout=0.1):
        super().__init__()
        
        # Causal depthwise conv: captures local spectro-temporal patterns
        self.pre = nn.Conv1d(input_dim, input_dim, kernel_size=3, padding=0, groups=input_dim)
        self.norm = nn.LayerNorm(input_dim)
        
        # Single GRU layer: models temporal evolution of noise
        self.gru = nn.GRU(input_dim, hidden_dim, batch_first=True)
        
        # Output layer: maps hidden state to noise power spectrum
        self.fc = nn.Linear(hidden_dim, input_dim)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, h=None):
        """
        Args:
            x: Input features (B, T, F) where F = input_dim
            h: Hidden state from previous time step (optional, for streaming)
        
        Returns:
            noise_est: Estimated noise power spectrum (B, T, F)
            h: Updated hidden state
        """
        # x: (B,T,F)
        x = x.transpose(1,2)              # (B,F,T)
        
        # Causal padding: pad (kernel_size-1) frames on the left only
        # This ensures the conv doesn't see future frames
        k = self.pre.kernel_size[0] if isinstance(self.pre.kernel_size, (list, tuple)) else self.pre.kernel_size
        pad_left = k - 1
        x = F.pad(x, (pad_left, 0))       # pad on time dimension (left, right)
        
        # Depthwise conv + normalization
        x = self.pre(x).transpose(1,2)    # local causal conv -> (B,T,F)
        x = self.norm(x)
        
        # GRU: temporal modeling
        out, h = self.gru(x, h)
        out = self.drop(out)
        
        # Output: noise power spectrum estimate
        # Use ReLU to ensure non-negative power values
        noise_est = F.relu(self.fc(out))
        
        return noise_est, h

    def count_parameters(self):
        """Count trainable parameters."""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [ ]:
# %% ------------------------- Cell 4: Load Datasets --------------------------
max_pairs = 1000
noise_files = load_wham_dataset(repo_root, mode="train", max_files=max_pairs)
clean_files = load_ears_dataset(repo_root, mode="train")
train_pairs = create_audio_pairs(noise_files, clean_files)
print(f"Train pairs: {len(train_pairs)}")

noise_val = load_wham_dataset(repo_root, mode="validation", max_files=150)
clean_val = load_ears_dataset(repo_root, mode="validation")
val_pairs = create_audio_pairs(noise_val, clean_val)
print(f"Val pairs: {len(val_pairs)}")

# Load TEST set (for final evaluation - never used during training)
noise_test = load_wham_dataset(repo_root, mode="test", max_files=150)
clean_test = load_ears_dataset(repo_root, mode="test")
test_pairs = create_audio_pairs(noise_test, clean_val)
print(f"Test pairs: {len(test_pairs)}")
print("\n [WARN] TEST SET: Only use for final evaluation, NOT during training/validation!")

In [ ]:
# %% ------------------------- Cell 5: Feature Extraction & Ground Truth -------------------------

def extract_noise_estimation_features(noisy_wave, noise_wave, fs, n_bands=32, frame_len=0.008, hop_len=0.004):
    """Extract features and ground truth for noise estimation training.
        
    References:
    - Cohen & Berdugo (2001) - "Noise estimation by minima controlled recursive averaging"
    - Gerkmann & Hendriks (2012) - "Unbiased MMSE-based noise power estimation"
    - Xu et al. (2014) - "Regression approach to speech enhancement"
    
    Args:
        noisy_wave: Noisy input signal (clean speech + noise)
        noise_wave: Pure noise signal (ground truth)
        fs: Sample rate (typically 16000 Hz)
        n_bands: Number of mel bands (default: 32)
        frame_len: Frame length in seconds (default: 0.008 = 8ms)
        hop_len: Hop length in seconds (default: 0.004 = 4ms)
    
    Returns:
        feats: (1, T, 2*n_bands) feature tensor [log-mel + delta]
        targets: (1, T, n_bands) ground truth noise power spectrum
    """
    # Convert to mono: handle both stereo and mono inputs
    if noisy_wave.dim() > 1 and noisy_wave.shape[0] > 1:
        noisy_wave = noisy_wave.mean(dim=0)
    elif noisy_wave.dim() > 1:
        noisy_wave = noisy_wave[0]
    
    if noise_wave.dim() > 1 and noise_wave.shape[0] > 1:
        noise_wave = noise_wave.mean(dim=0)
    elif noise_wave.dim() > 1:
        noise_wave = noise_wave[0]
    
    # Ensure both signals have same length (pad shorter one)
    max_len = max(noisy_wave.shape[0], noise_wave.shape[0])
    if noisy_wave.shape[0] < max_len:
        noisy_wave = F.pad(noisy_wave, (0, max_len - noisy_wave.shape[0]))
    if noise_wave.shape[0] < max_len:
        noise_wave = F.pad(noise_wave, (0, max_len - noise_wave.shape[0]))
    
    n_fft, hop = int(fs*frame_len), int(fs*hop_len)
    win = torch.hann_window(n_fft)

    def power_spectrum(w): 
        # center=False -> causal framing (no future samples)
        spec = torch.stft(w, n_fft, hop, window=win, center=False, return_complex=True)
        return spec.abs()**2

    mel_transform = torchaudio.transforms.MelScale(n_mels=n_bands, sample_rate=fs, n_stft=n_fft//2+1)
    
    # ============ FEATURES: Extract from NOISY audio (realistic input) ============
    mel_noisy = mel_transform(power_spectrum(noisy_wave)).clamp_min(1e-8)
    log_mel = torch.log(mel_noisy.T + 1e-8)  # (T, n_bands)
    
    # Delta features: temporal derivative (captures noise evolution)
    delta = torch.zeros_like(log_mel)
    delta[1:] = log_mel[1:] - log_mel[:-1]
    
    feats = torch.cat([log_mel, delta], 1).unsqueeze(0)  # (1, T, 2*n_bands)
    
    # ============ TARGETS: Extract from PURE NOISE (ground truth) ============
    mel_noise = mel_transform(power_spectrum(noise_wave))  # (n_bands, T)
    noise_power = mel_noise.T.clamp_min(1e-8)  # (T, n_bands)
    targets = noise_power.unsqueeze(0)  # (1, T, n_bands)
    
    # Ensure features and targets have same temporal dimension
    min_frames = min(feats.size(1), targets.size(1))
    feats = feats[:, :min_frames, :]
    targets = targets[:, :min_frames, :]
    
    return feats, targets

In [ ]:
# %% ------------------------- Cell 6: Dataset & Loader -----------------------
class NoiseEstimationDataset(torch.utils.data.Dataset):
    """Dataset for training TinyNoiseGru.
    
    Each sample:
    - Loads a clean speech file and a noise file
    - Mixes them at a random SNR
    - Extracts features from noisy mixture
    - Extracts ground truth from pure noise
    """
    def __init__(self, pairs, target_sr=16_000, snr_range=(-5, 15)):
        self.pairs = pairs
        self.sr = target_sr
        self.snr_range = snr_range
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        noise_path, clean_path = self.pairs[idx]
        snr = random.uniform(*self.snr_range)
        
        # preprocess_audio returns: clean_wave, noise_wave, noisy_wave, fs
        clean_wave, noise_wave, noisy_wave, fs = preprocess_audio(
            Path(clean_path), Path(noise_path), self.sr, snr, None
        )
        
        # Extract features and targets
        feats, targets = extract_noise_estimation_features(noisy_wave, noise_wave, fs)
        
        return feats.squeeze(0), targets.squeeze(0)


def collate_pad_noise(batch):
    """Collate function for variable-length sequences.
    
    Pads all sequences to the maximum length in the batch.
    """
    feats, targets = zip(*batch)
    lengths = [f.size(0) for f in feats]
    T_max = max(lengths)
    F_dim = feats[0].size(1)
    T_dim = targets[0].size(1)
    
    X = torch.zeros(len(batch), T_max, F_dim)
    Y = torch.zeros(len(batch), T_max, T_dim)
    
    for i, (f, t) in enumerate(zip(feats, targets)):
        X[i, :f.size(0)] = f
        Y[i, :t.size(0)] = t
    
    return X, Y, torch.tensor(lengths)


# Create datasets and loaders
train_ds = NoiseEstimationDataset(train_pairs)
val_ds = NoiseEstimationDataset(val_pairs)

train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=8, shuffle=True, 
    collate_fn=collate_pad_noise, pin_memory=True
)
val_dl = torch.utils.data.DataLoader(
    val_ds, batch_size=8, 
    collate_fn=collate_pad_noise, pin_memory=True
)

print(f"[INFO] Train batches: {len(train_dl)}")
print(f"[INFO] Val batches: {len(val_dl)}")

In [ ]:
# %% ------------------------- Cell 7: Training Loop -------------------------
from sklearn.metrics import mean_squared_error, mean_absolute_error
from IPython.display import clear_output, display
import matplotlib.pyplot as plt
import pandas as pd

# Initialize model
N_BANDS = 32  # Number of mel bands
model = TinyNoiseGru(input_dim=N_BANDS, hidden_dim=16).to(device)
print(f"[INFO] Model parameters: {model.count_parameters():,}")

# Optimizer and scheduler
opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-6)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=3)

# Loss function: MSE for regression
# Note: We'll compute per-frame loss and average over valid frames
crit = nn.MSELoss(reduction='none')

# Model saving paths
best_mse = float('inf')
bestPath = repo_root / "models" / "GRU_NoiseEst" / "tiny_noise_gru_best.pth"
bestPath.parent.mkdir(parents=True, exist_ok=True)

# Create directory for checkpoints
models_dir = repo_root / "models" / "GRU_NoiseEst" / "training_checkpoints"
models_dir.mkdir(parents=True, exist_ok=True)
print(f"[INFO] Checkpoint directory: {models_dir}")

# Create directory for training data
training_data_dir = repo_root / "results" / "GRU_NoiseEst" / "training_data"
training_data_dir.mkdir(parents=True, exist_ok=True)
print(f"[INFO] Training data directory: {training_data_dir}")

# Training configuration
num_epochs = 50  # Number of training epochs
early_stop_patience = 20
epochs_without_improvement = 0

# LR warmup function
def lr_warmup(epoch, warmup_epochs=5, base_lr=1e-3):
    """Linear warmup for first few epochs."""
    if epoch < warmup_epochs:
        return base_lr * (epoch + 1) / warmup_epochs
    return base_lr

# Prepare live plotting
plt.ion()
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
ax_loss, ax_mae, ax_rel, ax_lr = axes.flatten()

ax_loss.set_title("MSE Loss"); ax_loss.set_xlabel("Epoch"); ax_loss.set_ylabel("MSE"); ax_loss.grid(True, alpha=0.3)
ax_mae.set_title("MAE Loss"); ax_mae.set_xlabel("Epoch"); ax_mae.set_ylabel("MAE"); ax_mae.grid(True, alpha=0.3)
ax_rel.set_title("Relative Error"); ax_rel.set_xlabel("Epoch"); ax_rel.set_ylabel("Rel. Error (%)"); ax_rel.grid(True, alpha=0.3)
ax_lr.set_title("Learning Rate"); ax_lr.set_xlabel("Epoch"); ax_lr.set_ylabel("LR"); ax_lr.grid(True, alpha=0.3)

# Metrics storage
train_losses, val_losses = [], []
train_mae, val_mae = [], []
train_rel_err, val_rel_err = [], []
lr_history = []

print("="*80)
print("TRAINING: TinyNoiseGru - Lightweight Noise Power Spectrum Estimator")
print(f"Total Epochs: {num_epochs} | Early Stop Patience: {early_stop_patience}")
print("="*80)

for epoch in range(1, num_epochs + 1):
    # Apply LR warmup
    current_lr = lr_warmup(epoch, warmup_epochs=5)
    for param_group in opt.param_groups:
        param_group['lr'] = current_lr
    lr_history.append(current_lr)
    
    # ---------------- Train ----------------
    model.train()
    train_mse_total = 0.0
    train_mae_total = 0.0
    train_rel_total = 0.0
    train_samples = 0
    
    for x, y, lengths in tqdm(train_dl, desc=f"Epoch {epoch}/{num_epochs}", leave=False):
        x, y = x.to(device), y.to(device)
        
        # Forward pass
        noise_est, _ = model(x)
        
        # Compute loss only on valid (non-padded) frames
        B, T_max = noise_est.size(0), noise_est.size(1)
        mask = (torch.arange(T_max, device=lengths.device)[None, :] < lengths[:, None]).to(device)
        
        # MSE loss
        loss_per_frame = crit(noise_est, y).mean(dim=-1)  # Average over frequency bins
        loss = (loss_per_frame * mask).sum() / mask.sum().clamp_min(1.0)
        
        # Backward pass
        opt.zero_grad()
        loss.backward()
        
        # Gradient clipping (prevent exploding gradients)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        opt.step()
        
        # Track metrics
        with torch.no_grad():
            # MSE
            train_mse_total += loss.item() * mask.sum().item()
            
            # MAE
            mae_per_frame = torch.abs(noise_est - y).mean(dim=-1)
            train_mae_total += (mae_per_frame * mask).sum().item()
            
            # Relative error: |pred - true| / true
            rel_err = torch.abs(noise_est - y) / (y + 1e-8)
            train_rel_total += (rel_err.mean(dim=-1) * mask).sum().item()
            
            train_samples += mask.sum().item()
    
    # Average training metrics
    train_mse = train_mse_total / train_samples
    train_mae_epoch = train_mae_total / train_samples
    train_rel_epoch = (train_rel_total / train_samples) * 100  # Convert to percentage
    
    train_losses.append(train_mse)
    train_mae.append(train_mae_epoch)
    train_rel_err.append(train_rel_epoch)
    
    # ---------------- Validate ----------------
    model.eval()
    val_mse_total = 0.0
    val_mae_total = 0.0
    val_rel_total = 0.0
    val_samples = 0
    
    with torch.no_grad():
        for x, y, lengths in val_dl:
            x, y = x.to(device), y.to(device)
            
            # Forward pass
            noise_est, _ = model(x)
            
            # Compute metrics on valid frames
            B, T_max = noise_est.size(0), noise_est.size(1)
            mask = (torch.arange(T_max, device=lengths.device)[None, :] < lengths[:, None]).to(device)
            
            # MSE
            loss_per_frame = ((noise_est - y) ** 2).mean(dim=-1)
            val_mse_total += (loss_per_frame * mask).sum().item()
            
            # MAE
            mae_per_frame = torch.abs(noise_est - y).mean(dim=-1)
            val_mae_total += (mae_per_frame * mask).sum().item()
            
            # Relative error
            rel_err = torch.abs(noise_est - y) / (y + 1e-8)
            val_rel_total += (rel_err.mean(dim=-1) * mask).sum().item()
            
            val_samples += mask.sum().item()
    
    # Average validation metrics
    val_mse = val_mse_total / val_samples
    val_mae_epoch = val_mae_total / val_samples
    val_rel_epoch = (val_rel_total / val_samples) * 100
    
    val_losses.append(val_mse)
    val_mae.append(val_mae_epoch)
    val_rel_err.append(val_rel_epoch)
    
    # Update scheduler
    sched.step(val_mse)
    
    # ---------------- Save Best Model ----------------
    if val_mse < best_mse:
        best_mse = val_mse
        torch.save(model.state_dict(), bestPath)
        
        # Save checkpoint with epoch number
        checkpoint_path = models_dir / f"tiny_noise_gru_epoch_{epoch:03d}_mse_{val_mse:.6f}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        
        epochs_without_improvement = 0
        status = f"[INFO] NEW BEST (MSE={val_mse:.6f}) - Saved epoch {epoch}"
    else:
        epochs_without_improvement += 1
        status = f"[DEBUG] (No improvement: {epochs_without_improvement}/{early_stop_patience})"
    
    # ---------------- Display ----------------
    clear_output(wait=True)
    print(f"Epoch {epoch:2d}/{num_epochs}  LR={current_lr:.1e}")
    print(f"  Train: MSE={train_mse:.6f}  MAE={train_mae_epoch:.6f}  RelErr={train_rel_epoch:.2f}%")
    print(f"  Val:   MSE={val_mse:.6f}  MAE={val_mae_epoch:.6f}  RelErr={val_rel_epoch:.2f}%")
    print(f"  {status}")
    
    # Update plots
    for ax in axes.flatten():
        ax.cla()
    
    # MSE Loss plot
    ax_loss.plot(train_losses, "b-", label="Train MSE", linewidth=2)
    ax_loss.plot(val_losses, "r-", label="Val MSE", linewidth=2)
    ax_loss.legend(); ax_loss.set_xlabel("Epoch"); ax_loss.set_ylabel("MSE")
    ax_loss.set_title("MSE Loss"); ax_loss.grid(True, alpha=0.3)
    
    # MAE plot
    ax_mae.plot(train_mae, "b-", label="Train MAE", linewidth=2)
    ax_mae.plot(val_mae, "r-", label="Val MAE", linewidth=2)
    ax_mae.legend(); ax_mae.set_xlabel("Epoch"); ax_mae.set_ylabel("MAE")
    ax_mae.set_title("MAE Loss"); ax_mae.grid(True, alpha=0.3)
    
    # Relative Error plot
    ax_rel.plot(train_rel_err, "b-", label="Train Rel. Err", linewidth=2)
    ax_rel.plot(val_rel_err, "r-", label="Val Rel. Err", linewidth=2)
    ax_rel.legend(); ax_rel.set_xlabel("Epoch"); ax_rel.set_ylabel("Rel. Error (%)")
    ax_rel.set_title("Relative Error"); ax_rel.grid(True, alpha=0.3)
    
    # Learning Rate plot
    ax_lr.plot(lr_history, "g-", linewidth=2)
    ax_lr.set_xlabel("Epoch"); ax_lr.set_ylabel("LR")
    ax_lr.set_title("Learning Rate"); ax_lr.grid(True, alpha=0.3)
    ax_lr.set_yscale('log')
    
    plt.tight_layout()
    display(fig)
    plt.pause(0.001)
    
    # Early stopping
    if epochs_without_improvement >= early_stop_patience:
        print(f"\n[INFO] Early stopping: No improvement for {early_stop_patience} epochs")
        print(f"       Best MSE: {best_mse:.6f}")
        break
    
    # Secondary stopping: LR floor
    if current_lr < 1e-5:
        print(f"\n[INFO] LR floor reached: Stopping training")
        break

# Save training metrics to CSV
print("\n[INFO] Saving training metrics to CSV...")
metrics_df = pd.DataFrame({
    'epoch': list(range(1, len(train_losses) + 1)),
    'train_mse': train_losses,
    'val_mse': val_losses,
    'train_mae': train_mae,
    'val_mae': val_mae,
    'train_rel_err': train_rel_err,
    'val_rel_err': val_rel_err,
    'learning_rate': lr_history[:len(train_losses)]
})

csv_path = training_data_dir / "tiny_noise_gru_training_metrics.csv"
metrics_df.to_csv(csv_path, index=False)
print(f"[INFO] Metrics saved to: {csv_path}")

# Save final plot
plt.ioff()
fig_path = training_data_dir / "tiny_noise_gru_training_curves.png"
fig.savefig(fig_path, dpi=400, bbox_inches='tight')
print(f"[INFO] Training curves saved to: {fig_path}")

print("\n" + "="*80)
print(f"TRAINING COMPLETE - Best validation MSE: {best_mse:.6f}")
print(f"Model saved to: {bestPath}")
print("="*80)

## Evaluation & Visualization

Let's evaluate the trained model and visualize some predictions.

In [ ]:
# %% ------------------------- Cell 8: Evaluation on Test Set -------------------------
print("\n[INFO] Evaluating on TEST set...")

# Load best model
model.load_state_dict(torch.load(bestPath))
model.eval()

# Create test dataset
test_ds = NoiseEstimationDataset(test_pairs)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=8, 
    collate_fn=collate_pad_noise, pin_memory=True
)

test_mse_total = 0.0
test_mae_total = 0.0
test_rel_total = 0.0
test_samples = 0

with torch.no_grad():
    for x, y, lengths in tqdm(test_dl, desc="Testing"):
        x, y = x.to(device), y.to(device)
        
        # Forward pass
        noise_est, _ = model(x)
        
        # Compute metrics on valid frames
        B, T_max = noise_est.size(0), noise_est.size(1)
        mask = (torch.arange(T_max, device=lengths.device)[None, :] < lengths[:, None]).to(device)
        
        # MSE
        loss_per_frame = ((noise_est - y) ** 2).mean(dim=-1)
        test_mse_total += (loss_per_frame * mask).sum().item()
        
        # MAE
        mae_per_frame = torch.abs(noise_est - y).mean(dim=-1)
        test_mae_total += (mae_per_frame * mask).sum().item()
        
        # Relative error
        rel_err = torch.abs(noise_est - y) / (y + 1e-8)
        test_rel_total += (rel_err.mean(dim=-1) * mask).sum().item()
        
        test_samples += mask.sum().item()

# Average test metrics
test_mse = test_mse_total / test_samples
test_mae = test_mae_total / test_samples
test_rel_err = (test_rel_total / test_samples) * 100

print("\n" + "="*80)
print("TEST SET RESULTS")
print("="*80)
print(f"  MSE:              {test_mse:.6f}")
print(f"  MAE:              {test_mae:.6f}")
print(f"  Relative Error:   {test_rel_err:.2f}%")
print("="*80)

# Save test results
test_results = {
    'test_mse': test_mse,
    'test_mae': test_mae,
    'test_rel_err': test_rel_err
}

import json
results_path = training_data_dir / "tiny_noise_gru_test_results.json"
with open(results_path, 'w') as f:
    json.dump(test_results, f, indent=2)
print(f"\n[INFO] Test results saved to: {results_path}")

In [ ]:
# %% ------------------------- Cell 9: Visualize Predictions -------------------------
import matplotlib.pyplot as plt

# Get a random sample from test set
test_idx = random.randint(0, len(test_ds) - 1)
feats, targets = test_ds[test_idx]

# Add batch dimension and move to device
feats_batch = feats.unsqueeze(0).to(device)
targets_batch = targets.unsqueeze(0).to(device)

# Get prediction
model.eval()
with torch.no_grad():
    pred, _ = model(feats_batch)

# Move to CPU and remove batch dimension
pred = pred.squeeze(0).cpu().numpy()
targets = targets.squeeze(0).cpu().numpy()
feats = feats.cpu().numpy()

# Extract log-mel features (first half of feature vector)
log_mel = feats[:, :N_BANDS]

# Plot
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Plot 1: Input log-mel spectrogram
ax1 = axes[0]
im1 = ax1.imshow(log_mel.T, aspect='auto', origin='lower', cmap='viridis')
ax1.set_title("Input: Log-Mel Spectrogram (Noisy)")
ax1.set_xlabel("Frame")
ax1.set_ylabel("Mel Band")
plt.colorbar(im1, ax=ax1, label="Log Power")

# Plot 2: Ground truth noise power
ax2 = axes[1]
im2 = ax2.imshow(targets.T, aspect='auto', origin='lower', cmap='hot')
ax2.set_title("Ground Truth: Noise Power Spectrum")
ax2.set_xlabel("Frame")
ax2.set_ylabel("Mel Band")
plt.colorbar(im2, ax=ax2, label="Power")

# Plot 3: Predicted noise power
ax3 = axes[2]
im3 = ax3.imshow(pred.T, aspect='auto', origin='lower', cmap='hot')
ax3.set_title("Predicted: Noise Power Spectrum")
ax3.set_xlabel("Frame")
ax3.set_ylabel("Mel Band")
plt.colorbar(im3, ax=ax3, label="Power")

plt.tight_layout()
plt.show()

# Compute per-frame error
frame_mse = np.mean((pred - targets) ** 2, axis=1)
frame_mae = np.mean(np.abs(pred - targets), axis=1)

# Plot error over time
fig, axes = plt.subplots(2, 1, figsize=(14, 6))

ax1 = axes[0]
ax1.plot(frame_mse, 'b-', linewidth=1.5)
ax1.set_title("Per-Frame MSE")
ax1.set_xlabel("Frame")
ax1.set_ylabel("MSE")
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(frame_mae, 'r-', linewidth=1.5)
ax2.set_title("Per-Frame MAE")
ax2.set_xlabel("Frame")
ax2.set_ylabel("MAE")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Sample {test_idx} - Average MSE: {np.mean(frame_mse):.6f}, Average MAE: {np.mean(frame_mae):.6f}")

## Model Analysis

Let's analyze the model's performance characteristics.

In [ ]:
# %% ------------------------- Cell 10: Model Performance Analysis -------------------------

# Analyze model size and efficiency
total_params = model.count_parameters()
print(f"Total parameters: {total_params:,}")
print(f"Model size: {total_params * 4 / 1024:.2f} KB (float32)")

# Analyze computational complexity (FLOPs per frame)
# Rough estimate for hearing-aid deployment
print("\n[INFO] Computational Complexity Estimate:")
print(f"  - Conv1d: {N_BANDS * 3} FLOPs")
print(f"  - GRU: {3 * 16 * (N_BANDS + 16 + 1)} FLOPs")
print(f"  - FC: {16 * N_BANDS} FLOPs")
print(f"  - Total: ~{N_BANDS * 3 + 3 * 16 * (N_BANDS + 16 + 1) + 16 * N_BANDS} FLOPs per frame")
print(f"  - At 250 fps: ~{(N_BANDS * 3 + 3 * 16 * (N_BANDS + 16 + 1) + 16 * N_BANDS) * 250 / 1e6:.2f} MFLOPs/sec")

# Memory footprint for streaming (single frame processing)
print(f"\n[INFO] Memory Footprint:")
print(f"  - Model weights: {total_params * 4 / 1024:.2f} KB")
print(f"  - Hidden state: {16 * 4} bytes = {16 * 4 / 1024:.3f} KB")
print(f"  - Input buffer (3 frames): {N_BANDS * 3 * 4 / 1024:.3f} KB")
print(f"  - Total: ~{(total_params * 4 + 16 * 4 + N_BANDS * 3 * 4) / 1024:.2f} KB")
print("\n[SUCCESS] Model is lightweight enough for hearing-aid deployment!")

## Export Model for Inference

Export the trained model for use in speech enhancement algorithms.

In [ ]:
# %% ------------------------- Cell 11: Export Model -------------------------

# Save model architecture + weights in a single file
export_path = repo_root / "models" / "GRU_NoiseEst" / "tiny_noise_gru_complete.pth"

torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': {
        'input_dim': N_BANDS,
        'hidden_dim': 16,
        'dropout': 0.1
    },
    'training_config': {
        'n_bands': N_BANDS,
        'frame_len': 0.008,
        'hop_len': 0.004,
        'sample_rate': 16000
    },
    'performance': {
        'best_val_mse': best_mse,
        'test_mse': test_mse,
        'test_mae': test_mae,
        'test_rel_err': test_rel_err
    }
}, export_path)

print(f"[INFO] Complete model exported to: {export_path}")
print("\nTo load the model:")
print("```python")
print("checkpoint = torch.load('tiny_noise_gru_complete.pth')")
print("config = checkpoint['model_config']")
print("model = TinyNoiseGru(**config)")
print("model.load_state_dict(checkpoint['model_state_dict'])")
print("```")